In [ ]:
# 1. Imports and basic paths

from pathlib import Path
import json
from collections import Counter

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from ultralytics import YOLO
from PIL import Image

# Paths for your repo
DATA_DIR = Path("data")
ANN_DIR = DATA_DIR / "annotations"
IMG_DIR = DATA_DIR / "images"

VAL_JSON = ANN_DIR / "instances_val_gps_split_with_signs.json"
YOLO_MODEL_PATH = Path("best.pt")

print("Val json exists:", VAL_JSON.exists())
print("YOLO model exists:", YOLO_MODEL_PATH.exists())

Val json exists: True
YOLO model exists: True


In [2]:
# 2. Load COCO val and build image + category dicts

def load_coco(path: Path):
    with open(path, "r") as f:
        return json.load(f)

coco_val = load_coco(VAL_JSON)

print("Val images:", len(coco_val["images"]))
print("Val annotations:", len(coco_val["annotations"]))
print("Num categories:", len(coco_val["categories"]))

# image_id -> image_info
images_by_id = {img["id"]: img for img in coco_val["images"]}

# category_id -> name
cat_id_to_name = {c["id"]: c["name"] for c in coco_val["categories"]}
list(cat_id_to_name.items())

Val images: 2098
Val annotations: 21261
Num categories: 49


[(1, 'Police Officer'),
 (2, 'Police Vehicle'),
 (3, 'Cone'),
 (4, 'Fence'),
 (5, 'Drum'),
 (6, 'Barricade'),
 (7, 'Barrier'),
 (8, 'Work Vehicle'),
 (9, 'Vertical Panel'),
 (10, 'Tubular Marker'),
 (11, 'Arrow Board'),
 (12, 'Bike Lane'),
 (13, 'Work Equipment'),
 (14, 'Worker'),
 (15, 'Other Roadwork Objects'),
 (16, 'Temporary Traffic Control Message Board'),
 (17, 'Temporary Traffic Control Sign'),
 (19, 'Temporary Traffic Control Sign: left arrow'),
 (20, 'Temporary Traffic Control Sign: right arrow'),
 (21, 'Temporary Traffic Control Sign: up arrow'),
 (22, 'Temporary Traffic Control Sign: left chevron'),
 (23, 'Temporary Traffic Control Sign: right lane ends sign'),
 (24, 'Temporary Traffic Control Sign: two lane shift arrows'),
 (25, 'Temporary Traffic Control Sign: right chevron'),
 (26, 'Temporary Traffic Control Sign: lane shift arrow'),
 (27, 'Temporary Traffic Control Sign: up diagonal right arrow'),
 (28, 'Temporary Traffic Control Sign: left lane ends sign'),
 (29, 'Temp

In [12]:
from ultralytics import YOLO
model = YOLO(YOLO_MODEL_PATH)

print(model.names)


{0: 'Police Officer', 1: 'Police Vehicle', 2: 'Cone', 3: 'Fence', 4: 'Drum', 5: 'Barricade', 6: 'Barrier', 7: 'Work Vehicle', 8: 'Vertical Panel', 9: 'Tubular Marker', 10: 'Arrow Board', 11: 'Bike Lane', 12: 'Work Equipment', 13: 'Worker', 14: 'Other Roadwork Objects', 15: 'Temporary Traffic Control Message Board', 16: 'Temporary Traffic Control Sign', 17: 'Temporary Traffic Control Sign: left arrow', 18: 'Temporary Traffic Control Sign: right arrow', 19: 'Temporary Traffic Control Sign: up arrow', 20: 'Temporary Traffic Control Sign: left chevron', 21: 'Temporary Traffic Control Sign: right lane ends sign', 22: 'Temporary Traffic Control Sign: two lane shift arrows', 23: 'Temporary Traffic Control Sign: right chevron', 24: 'Temporary Traffic Control Sign: lane shift arrow', 25: 'Temporary Traffic Control Sign: up diagonal right arrow', 26: 'Temporary Traffic Control Sign: left lane ends sign', 27: 'Temporary Traffic Control Sign: bent left arrow', 28: 'Temporary Traffic Control Sign: 

In [13]:
import json, pathlib

ann = json.load(open("annotations/train.json"))

cats = {cat["id"]: cat["name"] for cat in ann["categories"]}
print(len(cats), cats)

FileNotFoundError: [Errno 2] No such file or directory: 'annotations/train.json'

In [3]:
# 3. Map scene_level_tags.travel_alteration to a simple scene label

def get_scene_label(img_dict):
    """
    Use scene_level_tags.travel_alteration to get a coarse scene label.
    lane_shift or workzone_ahead or None.
    """
    tags = img_dict.get("scene_level_tags", {})
    travel_alteration = tags.get("travel_alteration", [])

    # sometimes list, sometimes string
    if isinstance(travel_alteration, list):
        travel_alteration = travel_alteration[0] if travel_alteration else None

    if travel_alteration is None:
        return None

    txt = str(travel_alteration).lower()

    if "lane shift" in txt:
        return "lane_shift"

    # anything else non empty is considered generic workzone ahead
    return "workzone_ahead"

# quick check of distribution
scene_labels = [get_scene_label(img) for img in coco_val["images"]]
from collections import Counter
print("Scene label counts:", Counter([s for s in scene_labels if s is not None]))

Scene label counts: Counter({'workzone_ahead': 2041, 'lane_shift': 56})


In [4]:
# 4. Define semantic groups for categories

CHANNELIZATION = {
    "Cone",
    "Drum",
    "Barricade",
    "Barrier",
    "Vertical Panel",
    "Tubular Marker",
    "Fence",
}

WORKERS = {
    "Worker",
    "Police Officer",
}

VEHICLES = {
    "Work Vehicle",
    "Police Vehicle",
}

MESSAGE_BOARD = {
    "Temporary Traffic Control Message Board",
    "Arrow Board",
}

TTC_SIGNS = {
    "Temporary Traffic Control Sign",
    "Temporary Traffic Control Sign: left arrow",
    "Temporary Traffic Control Sign: right arrow",
    "Temporary Traffic Control Sign: up arrow",
    "Temporary Traffic Control Sign: left chevron",
    "Temporary Traffic Control Sign: right lane ends sign",
    "Temporary Traffic Control Sign: two lane shift arrows",
    "Temporary Traffic Control Sign: right chevron",
    "Temporary Traffic Control Sign: lane shift arrow",
    "Temporary Traffic Control Sign: up diagonal right arrow",
    "Temporary Traffic Control Sign: left lane ends sign",
    "Temporary Traffic Control Sign: bent left arrow",
    "Temporary Traffic Control Sign: flagger",
    "Temporary Traffic Control Sign: bent right arrow",
    "Temporary Traffic Control Sign: no left turn",
    "Temporary Traffic Control Sign: pedestrian: right arrow",
    "Temporary Traffic Control Sign: pedestrian: left arrow",
    "Temporary Traffic Control Sign: up diagonal left arrow",
    "Temporary Traffic Control Sign: pedestrian",
    "Temporary Traffic Control Sign: no right turn",
    "Temporary Traffic Control Sign: bi-directional arrow",
    "Temporary Traffic Control Sign: two upward diagonal arrows",
    "Temporary Traffic Control Sign: curved right arrow",
    "Temporary Traffic Control Sign: down diagonal left arrow",
    "Temporary Traffic Control Sign: do not enter sign",
    "Temporary Traffic Control Sign: worker",
    "Temporary Traffic Control Sign: bicycle",
    "Temporary Traffic Control Sign: two downward diagonal arrows",
    "Temporary Traffic Control Sign: curved left arrow",
    "Temporary Traffic Control Sign: curved left arrow, curved right arrow",
    "Temporary Traffic Control Sign: work vehicle",
    "Temporary Traffic Control Sign: traffic signal",
    "Temporary Traffic Control Sign: up arrow. stop sign",
}

OTHER_ROADWORK = {
    "Work Equipment",
    "Other Roadwork Objects",
}

name_to_group = {}
for n in CHANNELIZATION:
    name_to_group[n] = "channelization"
for n in WORKERS:
    name_to_group[n] = "workers"
for n in VEHICLES:
    name_to_group[n] = "vehicles"
for n in MESSAGE_BOARD:
    name_to_group[n] = "message_board"
for n in TTC_SIGNS:
    name_to_group[n] = "ttc_signs"
for n in OTHER_ROADWORK:
    name_to_group[n] = "other_roadwork"

unmapped = [n for n in cat_id_to_name.values() if n not in name_to_group]
print("Unmapped category names:", unmapped)

Unmapped category names: ['Bike Lane']


In [5]:
# 5. Load YOLOv12 model

yolo_model = YOLO(str(YOLO_MODEL_PATH))
yolo_model.to("cuda")  # or "cpu" if needed
yolo_model.eval()

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_runnin

In [6]:
# 6. Helpers to bin objects in left/mid/right and near/mid/far

def compute_position_bins(x_center_norm):
    """
    x_center_norm in [0, 1].
    Returns 'left', 'mid', or 'right'.
    """
    if x_center_norm < 1 / 3:
        return "left"
    elif x_center_norm < 2 / 3:
        return "mid"
    else:
        return "right"


def area_bin(area_norm):
    """
    area_norm is box_area / image_area.
    Rough depth bin: far, mid, near.
    """
    if area_norm < 0.02:
        return "far"
    elif area_norm < 0.08:
        return "mid"
    else:
        return "near"

In [7]:
# 7. Run YOLO on each val image and build feature rows

features = []

for img_info in tqdm(coco_val["images"], desc="Processing val images"):
    img_id = img_info["id"]
    file_name = img_info["file_name"]
    img_path = IMG_DIR / file_name

    if not img_path.exists():
        # if something is missing on disk, skip
        continue

    # run YOLO on this image
    results = yolo_model.predict(
        source=str(img_path),
        imgsz=960,     # can change later if you want
        conf=0.25,
        iou=0.7,
        verbose=False,
        device=0,
    )
    r = results[0]

    w = img_info["width"]
    h = img_info["height"]
    img_area = float(w * h)

    if r.boxes is None or len(r.boxes) == 0:
        # no detections, just zero features
        row = {
            "image_id": img_id,
            "file_name": file_name,
            "total_objs": 0,
        }
        for g in ["channelization", "workers", "vehicles", "ttc_signs", "message_board", "other_roadwork"]:
            row[f"count_{g}"] = 0
            row[f"frac_{g}"] = 0.0
        for side in ["left", "mid", "right"]:
            row[f"workers_{side}"] = 0
            row[f"channelization_{side}"] = 0
        for dist in ["near", "mid", "far"]:
            row[f"workers_{dist}"] = 0
            row[f"channelization_{dist}"] = 0
        row["scene_label"] = get_scene_label(img_info)
        features.append(row)
        continue

    # there are detections
    total_objs = 0
    group_counts = Counter()
    workers_side = Counter()
    channel_side = Counter()
    workers_dist = Counter()
    channel_dist = Counter()

    for box in r.boxes:
        cls_id = int(box.cls.item())
        # YOLO classes are 0 based, your COCO ids start at 1 and are in the same order
        cat_name = cat_id_to_name.get(cls_id + 1, None)
        if cat_name is None:
            continue

        group = name_to_group.get(cat_name, None)
        if group is None:
            group = "other_roadwork"

        total_objs += 1
        group_counts[group] += 1

        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cx = (x1 + x2) / 2.0
        cy = (y1 + y2) / 2.0
        area = (x2 - x1) * (y2 - y1)
        x_norm = cx / w
        area_norm = area / img_area

        side = compute_position_bins(x_norm)
        dist = area_bin(area_norm)

        if group == "workers":
            workers_side[side] += 1
            workers_dist[dist] += 1
        if group == "channelization":
            channel_side[side] += 1
            channel_dist[dist] += 1

    row = {
        "image_id": img_id,
        "file_name": file_name,
        "total_objs": total_objs,
    }

    for g in ["channelization", "workers", "vehicles", "ttc_signs", "message_board", "other_roadwork"]:
        c = group_counts[g]
        row[f"count_{g}"] = c
        row[f"frac_{g}"] = c / total_objs if total_objs > 0 else 0.0

    for side in ["left", "mid", "right"]:
        row[f"workers_{side}"] = workers_side[side]
        row[f"channelization_{side}"] = channel_side[side]

    for dist in ["near", "mid", "far"]:
        row[f"workers_{dist}"] = workers_dist[dist]
        row[f"channelization_{dist}"] = channel_dist[dist]

    row["scene_label"] = get_scene_label(img_info)

    features.append(row)

# build DataFrame
df_features = pd.DataFrame(features)
df_features.head()

Processing val images:   0%|          | 0/2098 [00:00<?, ?it/s]

,image_id,file_name,total_objs,count_channelization,frac_channelization,count_workers,frac_workers,count_vehicles,frac_vehicles,count_ttc_signs,...,channelization_left,workers_mid,channelization_mid,workers_right,channelization_right,workers_near,channelization_near,workers_far,channelization_far,scene_label
0,1,columbus_ed065d9b86d545b2af0042a058e7e907_0000...,12,3,0.250000,0,0.000000,9,0.750000,0,...,0,0,0,0,0,0,1,0,2,workzone_ahead
1,2,columbus_ed065d9b86d545b2af0042a058e7e907_0000...,21,5,0.238095,15,0.714286,1,0.047619,0,...,0,0,0,8,2,0,0,15,5,workzone_ahead
2,3,columbus_ed065d9b86d545b2af0042a058e7e907_0000...,3,1,0.333333,0,0.000000,2,0.666667,0,...,0,0,1,0,0,0,0,0,0,workzone_ahead
3,4,columbus_ed065d9b86d545b2af0042a058e7e907_0000...,7,1,0.142857,0,0.000000,6,0.857143,0,...,0,0,0,0,0,0,0,0,1,workzone_ahead
4,5,columbus_ed065d9b86d545b2af0042a058e7e907_0000...,4,1,0.250000,0,0.000000,3,0.750000,0,...,0,0,0,0,1,0,0,0,1,workzone_ahead


In [8]:
# 8. Sanity checks and save CSV

print("Feature table shape:", df_features.shape)
print("Scene label distribution:")
print(df_features["scene_label"].value_counts(dropna=False))

print(df_features.describe().T.head(20))

OUT_PATH = Path("models") / "scene_features_val_from_yolo.csv"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
df_features.to_csv(OUT_PATH, index=False)
print("Saved features to:", OUT_PATH)

Feature table shape: (2098, 26)
Scene label distribution:
scene_label
workzone_ahead    2041
lane_shift          56
None                 1
Name: count, dtype: int64
                       count         mean         std  min     25%  \
image_id              2098.0  1049.500000  605.784753  1.0  525.25   
total_objs            2098.0     6.897045    4.846315  0.0    3.00   
count_channelization  2098.0     3.557197    3.566096  0.0    1.00   
frac_channelization   2098.0     0.518858    0.340018  0.0    0.25   
count_workers         2098.0     2.629647    3.324827  0.0    0.00   
frac_workers          2098.0     0.351062    0.334336  0.0    0.00   
count_vehicles        2098.0     0.710200    1.888625  0.0    0.00   
frac_vehicles         2098.0     0.102434    0.235005  0.0    0.00   
count_ttc_signs       2098.0     0.000000    0.000000  0.0    0.00   
frac_ttc_signs        2098.0     0.000000    0.000000  0.0    0.00   
count_message_board   2098.0     0.000000    0.000000  0.0    0.0

In [5]:
from ultralytics import YOLO
from pathlib import Path
import random

YOLO_MODEL_PATH = "workzone-yolo-v12/experiment-1/weights/best.pt"  # adjust if needed
IMG_DIR = Path("data/images")  # same as before

model = YOLO(YOLO_MODEL_PATH)
print("Class map from model:")
print(model.names)

Class map from model:
{0: 'Police Officer', 1: 'Police Vehicle', 2: 'Cone', 3: 'Fence', 4: 'Drum', 5: 'Barricade', 6: 'Barrier', 7: 'Work Vehicle', 8: 'Vertical Panel', 9: 'Tubular Marker', 10: 'Arrow Board', 11: 'Bike Lane', 12: 'Work Equipment', 13: 'Worker', 14: 'Other Roadwork Objects', 15: 'Temporary Traffic Control Message Board', 16: 'Temporary Traffic Control Sign', 17: 'Temporary Traffic Control Sign: left arrow', 18: 'Temporary Traffic Control Sign: right arrow', 19: 'Temporary Traffic Control Sign: up arrow', 20: 'Temporary Traffic Control Sign: left chevron', 21: 'Temporary Traffic Control Sign: right lane ends sign', 22: 'Temporary Traffic Control Sign: two lane shift arrows', 23: 'Temporary Traffic Control Sign: right chevron', 24: 'Temporary Traffic Control Sign: lane shift arrow', 25: 'Temporary Traffic Control Sign: up diagonal right arrow', 26: 'Temporary Traffic Control Sign: left lane ends sign', 27: 'Temporary Traffic Control Sign: bent left arrow', 28: 'Temporary 

In [6]:
from pathlib import Path
import json
import pandas as pd

DATA_DIR = Path("data")
ANN_DIR = DATA_DIR / "annotations"
IMG_DIR = DATA_DIR / "images"

TRAIN_JSON = ANN_DIR / "instances_train_gps_split_with_signs.json"
VAL_JSON   = ANN_DIR / "instances_val_gps_split_with_signs.json"

In [7]:
def load_coco(path: Path):
    with open(path, "r") as f:
        return json.load(f)

def build_img_and_anns(coco):
    imgs = {img["id"]: img for img in coco["images"]}
    anns_by_img = {img_id: [] for img_id in imgs.keys()}
    for ann in coco["annotations"]:
        anns_by_img[ann["image_id"]].append(ann)
    return imgs, anns_by_img

coco_train = load_coco(TRAIN_JSON)
coco_val   = load_coco(VAL_JSON)

train_imgs, train_anns = build_img_and_anns(coco_train)
val_imgs,   val_anns   = build_img_and_anns(coco_val)

len(train_imgs), len(val_imgs)

(5318, 2098)

In [8]:
def get_scene_label(img_dict):
    tags = img_dict.get("scene_level_tags", {})
    travel_alteration = tags.get("travel_alteration", [])

    if isinstance(travel_alteration, list):
        travel_alteration = travel_alteration[0] if travel_alteration else None

    if travel_alteration is None:
        return None

    txt = str(travel_alteration).lower()

    if "lane shift" in txt:
        return "lane_shift"
    if "workzone ahead" in txt or "work zone ahead" in txt:
        return "workzone_ahead"

    return None

def build_scene_df(imgs_dict, split_name):
    rows = []
    for img_id, img in imgs_dict.items():
        label = get_scene_label(img)
        if label is None:
            continue
        rows.append(
            {
                "image_id": img_id,
                "file_name": img["file_name"],
                "scene_label": label,
                "split": split_name,
            }
        )
    return pd.DataFrame(rows)

df_train = build_scene_df(train_imgs, "train")
df_val   = build_scene_df(val_imgs, "val")

print(df_train["scene_label"].value_counts())
print(df_val["scene_label"].value_counts())

scene_label
lane_shift    238
Name: count, dtype: int64
scene_label
lane_shift    56
Name: count, dtype: int64


In [9]:
def add_image_path(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["img_path"] = df["file_name"].apply(lambda fn: IMG_DIR / fn)
    exists_mask = df["img_path"].apply(lambda p: p.exists())
    missing = (~exists_mask).sum()
    if missing > 0:
        print(f"Warning: {missing} images not found on disk, dropping them.")
    return df[exists_mask].reset_index(drop=True)

df_train = add_image_path(df_train)
df_val   = add_image_path(df_val)

print(df_train.shape, df_val.shape)
df_val.head()


(238, 5) (56, 5)


,image_id,file_name,scene_label,split,img_path
0,129,boston_e8b7c866e5734e8b898ea791643bf1d8_000003...,lane_shift,val,data/images/boston_e8b7c866e5734e8b898ea791643...
1,131,boston_e8b7c866e5734e8b898ea791643bf1d8_000003...,lane_shift,val,data/images/boston_e8b7c866e5734e8b898ea791643...
2,132,boston_e8b7c866e5734e8b898ea791643bf1d8_000003...,lane_shift,val,data/images/boston_e8b7c866e5734e8b898ea791643...
3,185,boston_d7d96e38384140d89a8cb3c42c84dbea_000002...,lane_shift,val,data/images/boston_d7d96e38384140d89a8cb3c42c8...
4,186,boston_d7d96e38384140d89a8cb3c42c84dbea_000002...,lane_shift,val,data/images/boston_d7d96e38384140d89a8cb3c42c8...


In [3]:
from ultralytics import YOLO

YOLO_MODEL_PATH = "workzone-yolo-v12/experiment-1/weights/best.pt"  # adjust if needed
model = YOLO(YOLO_MODEL_PATH)
print(model.names)

def inspect_image_predictions(row, conf=0.25, iou=0.7):
    img_path = row["img_path"]
    print(f"\nImage id={row['image_id']}  file={row['file_name']}")
    print(f"Path: {img_path}")

    results = model.predict(
        source=str(img_path),
        conf=conf,
        iou=iou,
        verbose=False,
        device=0
    )
    r = results[0]

    if r.boxes is None or len(r.boxes) == 0:
        print("No detections")
        return

    cls = r.boxes.cls.cpu().numpy().astype(int)
    confs = r.boxes.conf.cpu().numpy()
    xyxy = r.boxes.xyxy.cpu().numpy()

    for cid, c, box in zip(cls, confs, xyxy):
        name = model.names.get(int(cid), f"class_{cid}")
        x1, y1, x2, y2 = box
        print(f"  {name:45s}  conf={c:.3f}  box=({x1:.1f},{y1:.1f},{x2:.1f},{y2:.1f})")


{0: 'Police Officer', 1: 'Police Vehicle', 2: 'Cone', 3: 'Fence', 4: 'Drum', 5: 'Barricade', 6: 'Barrier', 7: 'Work Vehicle', 8: 'Vertical Panel', 9: 'Tubular Marker', 10: 'Arrow Board', 11: 'Bike Lane', 12: 'Work Equipment', 13: 'Worker', 14: 'Other Roadwork Objects', 15: 'Temporary Traffic Control Message Board', 16: 'Temporary Traffic Control Sign', 17: 'Temporary Traffic Control Sign: left arrow', 18: 'Temporary Traffic Control Sign: right arrow', 19: 'Temporary Traffic Control Sign: up arrow', 20: 'Temporary Traffic Control Sign: left chevron', 21: 'Temporary Traffic Control Sign: right lane ends sign', 22: 'Temporary Traffic Control Sign: two lane shift arrows', 23: 'Temporary Traffic Control Sign: right chevron', 24: 'Temporary Traffic Control Sign: lane shift arrow', 25: 'Temporary Traffic Control Sign: up diagonal right arrow', 26: 'Temporary Traffic Control Sign: left lane ends sign', 27: 'Temporary Traffic Control Sign: bent left arrow', 28: 'Temporary Traffic Control Sign: 

In [10]:
# Pick some random val images
sample_rows = df_val.sample(10, random_state=0)

for _, row in sample_rows.iterrows():
    inspect_image_predictions(row, conf=0.25)



Image id=1915  file=denver_ebad84687fa141e2abe66a65753da0ac_000002_14640.jpg
Path: data/images/denver_ebad84687fa141e2abe66a65753da0ac_000002_14640.jpg
  Work Vehicle                                   conf=0.956  box=(472.0,419.6,754.3,814.8)
  Police Officer                                 conf=0.901  box=(278.7,748.8,383.9,950.3)
  Police Officer                                 conf=0.870  box=(750.3,692.4,795.4,798.9)
  Police Officer                                 conf=0.842  box=(871.9,673.0,902.6,742.0)
  Police Officer                                 conf=0.725  box=(753.4,648.0,771.9,688.6)
  Police Officer                                 conf=0.664  box=(865.9,648.8,880.2,686.8)
  Police Officer                                 conf=0.599  box=(653.9,648.6,670.3,687.9)
  Police Officer                                 conf=0.550  box=(567.1,669.5,586.2,720.1)
  Barricade                                      conf=0.468  box=(696.6,622.1,791.6,685.8)
  Barricade                 

In [11]:
from collections import Counter
from ultralytics import YOLO
from tqdm.auto import tqdm

YOLO_MODEL_PATH = "workzone-yolo-v12/experiment-1/weights/best.pt"  # adjust if needed
model = YOLO(YOLO_MODEL_PATH)

# make sure df_val exists and has 'img_path'
print(len(df_val), "val images")

det_counts = Counter()
conf_thr = 0.15  # lower conf to catch rare TTC signs

for _, row in tqdm(df_val.iterrows(), total=len(df_val), desc="Scanning val detections"):
    img_path = row["img_path"]
    results = model.predict(
        source=str(img_path),
        conf=conf_thr,
        iou=0.7,
        verbose=False,
        device=0,
    )
    r = results[0]
    if r.boxes is None or len(r.boxes) == 0:
        continue
    cls_ids = r.boxes.cls.cpu().numpy().astype(int)
    for cid in cls_ids:
        det_counts[int(cid)] += 1

det_counts


56 val images


Scanning val detections:   0%|          | 0/56 [00:00<?, ?it/s]

Counter({0: 253, 5: 81, 4: 69, 6: 41, 2: 38, 9: 29, 1: 25, 7: 23, 3: 15, 8: 4})

In [12]:
from ultralytics import YOLO

YOLO_MODEL_PATH = "workzone-yolo-v12/experiment-1/weights/best.pt"  # adjust if needed
model = YOLO(YOLO_MODEL_PATH)

results = model.val(
    data="workzone_yolo/workzone_yolo.yaml",
    imgsz=960,
    conf=0.25,
    iou=0.7,
    save_json=True,            # needed for COCO-style stats
    plots=True,                # THIS generates confusion matrix image
    device=0
)


Ultralytics 8.3.232 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti Laptop GPU, 12227MiB)
YOLOv12s summary (fused): 159 layers, 9,249,843 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 230.9±36.2 MB/s, size: 670.0 KB)
val: Scanning /home/wesley/code/workzone/workzone_yolo/labels/val.cache... 2046 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2046/2046 1.0Mit/s 0.0s0s
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000000_07230.jpg: 14 duplicate labels removed
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000000_07410.jpg: 13 duplicate labels removed
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000000_08220.jpg: 9 duplicate labels removed
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000001_08160.jpg: 8 duplicate labe

In [14]:
from ultralytics import YOLO

model = YOLO("workzone-yolo-v12/experiment-1/weights/best.pt")
metrics = model.val(data="workzone_yolo/workzone_yolo.yaml", split="val", verbose=False)

print("num classes in metrics:", len(metrics.names))
print("names dict:", metrics.names)

Ultralytics 8.3.232 🚀 Python-3.12.3 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti Laptop GPU, 12227MiB)
YOLOv12s summary (fused): 159 layers, 9,249,843 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2161.8±951.1 MB/s, size: 622.6 KB)
val: Scanning /home/wesley/code/workzone/workzone_yolo/labels/val.cache... 2046 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2046/2046 1.4Mit/s 0.0s0s
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000000_07230.jpg: 14 duplicate labels removed
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000000_07410.jpg: 13 duplicate labels removed
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000000_08220.jpg: 9 duplicate labels removed
val: /home/wesley/code/workzone/workzone_yolo/images/val/houston_1313e092c4d94bdf8349c99f763354b4_000001_08160.jpg: 8 duplicate la

In [20]:
from pathlib import Path

img_path = Path("data/demo/test2.jpg")  # one with TTC sign
res = model(img_path, conf=0.25)[0]

for box in res.boxes:
    cls_id = int(box.cls.item())
    name = model.names[cls_id]
    print(cls_id, name, float(box.conf))


image 1/1 /home/wesley/code/workzone/data/demo/test2.jpg: 640x960 1 Work Vehicle, 2 Tubular Markers, 65.7ms
Speed: 16.1ms preprocess, 65.7ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 960)
7 Work Vehicle 0.9540051221847534
9 Tubular Marker 0.5415558815002441
9 Tubular Marker 0.2876492738723755


In [16]:
# High-level TTC groups using your model.names indices
TTC_GROUPS = {
    # General TTC devices
    "message_board": [15],                       # Temporary Traffic Control Message Board
    "generic_ttc_sign": [16],                    # Temporary Traffic Control Sign (unspecified)
    "arrow_board": [10],                         # Arrow Board

    # Lane configuration / closure
    "lane_shift": [22, 24, 27, 29, 37, 45],      # shift arrows, two lane shift arrows, bent arrows, two upward arrows, paired curved arrows
    "lane_ends": [21, 26],                       # right lane ends, left lane ends
    "lane_merge_diverge": [38, 44, 39, 43],      # curved left/right arrows, diagonal arrows

    # Worker / construction warnings
    "worker_warning": [28, 41],                  # flagger, worker sign
    "work_vehicle_sign": [46],                   # work vehicle sign

    # Pedestrian / bicycle control
    "pedestrian": [31, 32, 33, 34],              # ped arrows + generic ped sign
    "bicycle": [42],                             # bicycle sign

    # Turn / entry control, misc regulatory
    "turn_restriction": [30, 35],                # no left turn, no right turn
    "entry_control": [40, 48],                   # do not enter, up arrow + stop sign
}

In [17]:
import numpy as np

def summarize_ttc_groups(result, ttc_groups=TTC_GROUPS):
    """
    result: one Ultralytics YOLO result object (from model(img_path)[0])
    Returns: dict {group_name: count}
    """
    cls_ids = result.boxes.cls.cpu().numpy().astype(int)
    counts = {g: 0 for g in ttc_groups.keys()}

    for cid in cls_ids:
        for group, idx_list in ttc_groups.items():
            if cid in idx_list:
                counts[group] += 1

    # drop empty groups for a cleaner print
    return {g: c for g, c in counts.items() if c > 0}


In [21]:
img_path = "data/demo/test1.jpg"
res = model(img_path, conf=0.25)[0]

print("Raw classes:", [model.names[int(c)] for c in res.boxes.cls])
print("TTC group summary:", summarize_ttc_groups(res))



image 1/1 /home/wesley/code/workzone/data/demo/test1.jpg: 640x960 3 Police Officers, 2 Police Vehicles, 1 Cone, 1 Tubular Marker, 72.1ms
Speed: 9.0ms preprocess, 72.1ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 960)
Raw classes: ['Police Vehicle', 'Cone', 'Tubular Marker', 'Police Officer', 'Police Officer', 'Police Vehicle', 'Police Officer']
TTC group summary: {}


In [25]:
from ultralytics import YOLO
import matplotlib.pyplot as plt

YOLO_MODEL_PATH = "workzone-yolo-v12/experiment-1/weights/best.pt"
IMG_PATH = "data/demo/test2.jpg"

model = YOLO(YOLO_MODEL_PATH)

results = model(IMG_PATH, conf=0.25)  # run inference
res = results[0]

annotated = res.plot()  # numpy array with boxes drawn

plt.figure(figsize=(8, 10))
plt.imshow(annotated)
plt.axis("off")
plt.tight_layout()
plt.show()

print("Predicted classes:", [model.names[int(c)] for c in res.boxes.cls])


image 1/1 /home/wesley/code/workzone/data/demo/test2.jpg: 640x960 1 Work Vehicle, 2 Tubular Markers, 27.5ms
Speed: 4.8ms preprocess, 27.5ms inference, 3.8ms postprocess per image at shape (1, 3, 640, 960)


<Figure size 800x1000 with 1 Axes>

Predicted classes: ['Work Vehicle', 'Tubular Marker', 'Tubular Marker']


In [28]:
from ultralytics import YOLO

YOLO_MODEL_PATH = "workzone-yolo-v12/experiment-1/weights/best.pt"
IMG_PATH = "data/demo/test1.jpg"

model = YOLO(YOLO_MODEL_PATH)

results = model(
    source=IMG_PATH,
    conf=0.25,
    save=True,
    project="workzone-debug",
    name="tcc_test",
    exist_ok=True,
)

print("Saved to:", results[0].save_dir)



image 1/1 /home/wesley/code/workzone/data/demo/test1.jpg: 640x960 3 Police Officers, 2 Police Vehicles, 1 Cone, 1 Tubular Marker, 43.0ms
Speed: 3.3ms preprocess, 43.0ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 960)
Results saved to /home/wesley/code/workzone/workzone-debug/tcc_test
Saved to: /home/wesley/code/workzone/workzone-debug/tcc_test
